## Run zonal statistics for climate datasets hosted on AWS

Import libraries

In [139]:
from os.path import join, expanduser
import os
import boto3
from rasterstats import zonal_stats
import geopandas as gpd
from botocore.config import Config
from botocore import UNSIGNED
import rasterio as rio
import geopandas as gpd
from rasterio import features
from rasterio.plot import show
from affine import Affine
from shapely.geometry import box
import pandas as pd
from tqdm import tqdm

Subnational boundaries have been curated from OCHA via the Humanitarian Data Exchange. 

In [5]:
adm1 = gpd.read_file(join(expanduser("~"), 'data', 'pacific', 'admin', 'PIC_adm1.shp'))
adm2 = gpd.read_file(join(expanduser("~"), 'data', 'pacific', 'admin', 'PIC_adm2.shp'))

In [6]:
client = boto3.client('s3')
paginator = client.get_paginator('list_objects')

List available datasets

In [7]:
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix='climate/products/', Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

climate/products/cdd-imerg/
climate/products/cwd-imerg/
climate/products/drydays-imerg/
climate/products/spei-terraclimate/
climate/products/spi-chirps/
climate/products/spi-terraclimate/
climate/products/wetdays-imerg/


List directories within data folder

In [8]:
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix='climate/products/cdd-imerg/', Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

climate/products/cdd-imerg/cdd_1mm/
climate/products/cdd-imerg/cdd_5mm/
climate/products/cdd-imerg/cdd_annual_max_1mm/
climate/products/cdd-imerg/cdd_annual_max_5mm/
climate/products/cdd-imerg/cdd_monthly_max_1mm/
climate/products/cdd-imerg/cdd_monthly_max_5mm/


List and count raster files  
**Consecutive Dry Days, 1mm**

In [82]:
dataset = 'cdd-imerg'
indicator = 'cdd_monthly_max'
indicator_type = '1mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

In [13]:
n = 0
for page in result:
    for item in page['Contents']:
        fname = item['Key']
        if fname.endswith('.tif'):
            print(fname)
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-8:fname.find('.tif')]
            n+=1
#             data_rio = rio.open(url)
n

climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200006.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200007.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200008.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200009.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200010.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200011.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200012.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200101.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200102.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg_200103.tif
climate/products/cdd-imerg/cdd_monthly_max_1mm/wld_cli_cdd_1mm_monthly_max_imerg

For each raster file, iterate through each admin 1 polygon, extract raster data, and run zonal statistics function

In [22]:
dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
            print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
            print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

  0%|          | 0/260 [00:00<?, ?it/s]

processing cdd_monthly_max_1mm, for 200006


  1%|          | 2/260 [00:05<12:16,  2.86s/it]

	no intersect for 25 polygons
processing cdd_monthly_max_1mm, for 200007


  1%|          | 3/260 [00:10<16:20,  3.81s/it]

...

	no intersect for 25 polygons
processing cdd_monthly_max_1mm, for 202111


100%|█████████▉| 259/260 [25:16<00:05,  5.29s/it]

	no intersect for 25 polygons
processing cdd_monthly_max_1mm, for 202112


100%|██████████| 260/260 [25:22<00:00,  5.86s/it]

	no intersect for 25 polygons


In [69]:
out_dir = join(expanduser("~"), 'data', 'pacific', 'output', 'climate')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [70]:
df_merge = pd.concat(dfs)

In [79]:
df_join = adm1[['ADM0_NAME', 'ADM0_PCODE', 'ADM1_NAME', 'ADM1_PCODE']].join(df_merge)

In [80]:
df_join.reset_index(inplace=True)

In [83]:
df_join.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

**CDD 5 mm**

In [86]:
dataset = 'cdd-imerg'
indicator = 'cdd_monthly_max'
indicator_type = '5mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

In [87]:
dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

100%|██████████| 260/260 [26:10<00:00,  6.04s/it]


In [88]:
df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

**CWD 1 mm**

In [108]:
dataset = 'cwd-imerg'
indicator = 'cwd_monthly_max'
indicator_type = '1mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [25:09<00:00,  5.81s/it]


**CWD 5 mm**

In [109]:
dataset = 'cwd-imerg'
indicator = 'cwd_monthly_max'
indicator_type = '5mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [24:41<00:00,  5.70s/it]


**Dry Days 1mm**

In [110]:
dataset = 'drydays-imerg'
indicator = 'drydays_monthly_sum'
indicator_type = '1mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [25:08<00:00,  5.80s/it]


**Dry Days 5mm**

In [111]:
dataset = 'drydays-imerg'
indicator = 'drydays_monthly_sum'
indicator_type = '5mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [25:09<00:00,  5.81s/it]


**Wet Days 1mm**

In [112]:
dataset = 'wetdays-imerg'
indicator = 'wetdays_monthly_sum'
indicator_type = '1mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [25:03<00:00,  5.78s/it]


**Wet Days 5mm**

In [138]:
dataset = 'wetdays-imerg'
indicator = 'wetdays_monthly_sum'
indicator_type = '5mm'
prefix = indicator+"_"+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-6:fname.find('.tif')]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 260/260 [25:10<00:00,  5.81s/it]


**SPEI 03**

In [113]:
dataset = 'spei-terraclimate/geotiff'
indicator = 'spei'
indicator_type = '03'
prefix = indicator+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-10:fname.find('.tif')-3]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 536/536 [55:30<00:00,  6.21s/it] 


**SPEI 12**

In [114]:
dataset = 'spei-terraclimate/geotiff'
indicator = 'spei'
indicator_type = '12'
prefix = indicator+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-10:fname.find('.tif')-3]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 536/536 [57:52<00:00,  6.48s/it] 


**SPI 03**

In [115]:
dataset = 'spi-chirps/geotiff'
indicator = 'spi'
indicator_type = '03'
prefix = indicator+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-10:fname.find('.tif')-3]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 981/981 [1:22:01<00:00,  5.02s/it]


**SPI 12**

In [116]:
dataset = 'spi-chirps/geotiff'
indicator = 'spi'
indicator_type = '12'
prefix = indicator+indicator_type
result = paginator.paginate(Bucket='wbgdecinternal-ntl', Prefix=f'climate/products/{dataset}/{prefix}/', Delimiter='/')

dfs = []
for page in result:
    
    # iterate through raster files
    for item in tqdm(page['Contents']):
        fname = item['Key']
        if fname.endswith('.tif'):
            url = 's3://wbgdecinternal-ntl/' + fname
            date = fname[fname.find('.tif')-10:fname.find('.tif')-3]
#             print(f"processing {prefix}, for {date}")
            data_rio = rio.open(url)
            
            # iterate through adm polygons
            res = {}
            fail = 0
            for idx, row in adm1.iterrows():
                try:
                    window = features.geometry_window(data_rio, [row.geometry])
                    ul_y = window.col_off
                    lr_x = window.row_off
                    t = data_rio.transform
                    shifted_affine = Affine(t.a, t.b, t.c+ul_y*t.a, t.d, t.e, t.f+lr_x*t.e)
                    data = data_rio.read(1, window=window)
                    zs = zonal_stats(row.geometry, data, affine=shifted_affine, nodata=data_rio.nodata, stats=['median','max', 'mean', 'std'], prefix=prefix+"_")
                    res[idx] = zs[0]
                except:
                    fail+=1
                    res[idx] = {
                        f'{prefix}_max': pd.NA,
                        f'{prefix}_mean': pd.NA,
                        f'{prefix}_std': pd.NA,
                        f'{prefix}_median': pd.NA
                    }
#                 print(f"done with {idx}")
#             print(f"\tno intersect for {fail} polygons")
            df = pd.DataFrame(res).transpose()
            df.loc[:, "date"] = date
            dfs.append(df)

df_merge = pd.concat(dfs)
df_merge.reset_index(inplace=True)
df_merge.to_csv(os.path.join(out_dir, f'adm1_{prefix}.csv'))

100%|██████████| 963/963 [1:18:07<00:00,  4.87s/it]
